# ĐỒ ÁN THỰC HÀNH
# CSC17104 – LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU

Giảng viên: Bùi Tiến Lên

Thông tin thành viên:
1. Nguyễn Văn Tuấn Đạt 19120472
2.
3. Luân Mã Khương 20120515
4. 

## Import

In [1]:
import numpy as np
import pandas as pd

## A. Thu thập dữ liệu
- Tên dữ liệu: HR Analytics: Job Change of Data Scientists
- Nguồn lấy dữ liệu: kaggle
- Link dữ liệu: https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists
- License: CC0: Public Domain
- Mô tả về dữ liệu: dữ liệu gồm tập train và tập test. Trong đồ án này, nhóm sẽ sử dụng tập train.
            + Dữ liệu gồm 14 cột và 19158 dòng.

In [2]:
df = pd.read_csv('aug_train.csv', encoding='utf-8')
df.shape

(19158, 14)

## B. Khám phá dữ liệu (thường đan xen với pha tiền xử lý dữ liệu)

Mỗi dòng có ý nghĩa gì? Có vấn đề với các dòng có ý nghĩa khác nhau không?

Mỗi cột có ý nghĩa gì?
            
            - enrollee_id : ID với từng ứng viên

            - city: Mã thành phố

            - city_development_index : Chỉ số phát triển của thành phố (theo tỷ lệ)

            - gender: Giới tính

            - relevent_experience: Kinh nghiệm liên quan của ứng viên

            - enrolled_university: Loại khóa học đại học đã đăng ký nếu có

            - education_level: Trình độ học vấn

            - major_discipline: Ngành học chính

            - experience: Kinh nghiệm (tính theo năm)

            - company_size: Số lượng nhân viên trong công ty của người sử dụng lao động hiện tại

            - company_type : Loại chủ lao động hiện tại

            - lastnewjob: Sự khác biệt về số năm giữa công việc trước đây và công việc hiện tại

            - training_hours: Thời gian hoàn thành huấn luyện

            - target: 0 – Không nhảy việc, 1 – Nhảy việc

Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

In [3]:
df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [4]:
df['experience'].replace({'1':'1-5',
                                '2':'1-5',
                                '3':'1-5',
                                '4':'1-5',
                                '5':'1-5',
                                '6':'6-10',
                                '7':'6-10',
                                '8':'6-10',
                                '9':'6-10',
                                '10':'6-10',
                                '11':'11-15',
                                '12':'11-15',
                                '13':'11-15',
                                '14':'11-15',
                                '15':'11-15',
                                '16':'16-20',
                                '17':'16-20',
                                '18':'16-20',
                                '19':'16-20',
                                '20':'16-20'},inplace=True)
df

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,11-15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,1-5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,11-15,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,11-15,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


Kiểm tra xem có dữ liệu nào bị trùng lặp hay không

In [5]:
remove_duplicates = df.drop_duplicates()
dup = remove_duplicates.shape == df.shape

if dup:
    print("Khong bi trung lap")
else:
    print("Co du lieu bi trung")

Khong bi trung lap


Phân nhóm theo giới tính của các ứng viên

In [6]:
applicants = df.copy()
applicants

male_applicants = applicants.loc[applicants['gender']=='Male']
print(male_applicants.shape)
#print(male_applicants)

female_applicants = applicants.loc[applicants['gender']=='Female']
print(female_applicants.shape)
#print(female_applicants)

other_gender_applicants = applicants.loc[pd.isna(applicants['gender'])]
other_gender_applicants = other_gender_applicants.append(applicants.loc[applicants['gender']=='Other'])
print(other_gender_applicants.shape)
#print(other_gender_applicants)

(13221, 14)
(1238, 14)
(4699, 14)


Ta nhận thấy số lượng ứng viên nam vượt trội hơn hẳn so với nữ và giới tính khác.<br>
Dù vậy, liệu những ứng viên nam này có ở thành phố có mức sống cao hơn hay không?

Phân tích điều kiện của các ứng viên

In [14]:
print(male_applicants['city_development_index'].mean())
print(female_applicants['city_development_index'].mean())
print(other_gender_applicants['city_development_index'].mean())

0.8390526435216701
0.8447399030694669
0.7959495637369653


Dựa theo số liệu trên ta thấy nữ giới ở các thành phố có mức sống cao hơn nam giới và các giới tính khác mặc dù số lượng ứng viên nam giới cao hơn nữ giới rất nhiều

Như vậy, liệu nữ giới có trình độ học vấn cao hơn nam giới và các giởi tính khác hay không?<br>
Ta cùng xem xét các khóa học họ đã tham gia

List ```male_enrolled_rate``` sẽ có 3 phần tử:
- 0: tỉ lệ tham gia khóa học full time
- 1: tỉ lệ tham gia khóa học part time
- 2: tỉ lệ không tham gia khóa học (no_enrollment + blanks)

In [23]:
def enrolled_rate(enrollment):
    enrolled_rate = []
    enrolled_rate.append(enrollment[1] / enrollment.sum())
    enrolled_rate.append(enrollment[2] / enrollment.sum())
    enrolled_rate.append((enrollment[0] + enrollment[3]) / enrollment.sum())
    return enrolled_rate
    

male_enrollment = male_applicants['enrolled_university'].value_counts(dropna=False)
female_enrollment = female_applicants['enrolled_university'].value_counts(dropna=False)
other_gender_enrollment = other_gender_applicants['enrolled_university'].value_counts(dropna=False)

print(male_enrolled_rate)
print(female_enrolled_rate)
print(other_gender_enrolled_rate)

[0.17487330761667044, 0.060207246047954015, 0.7649194463353756]
[0.17447495961227788, 0.05250403877221325, 0.7730210016155089]
[0.2615450095765056, 0.0717173866780166, 0.6667376037454777]


Như vậy ta thấy tỉ lệ tham gia các lớp học ở nam giới và nữ giới khá là giống nhau<br>
Bên cạnh đó, các ứng viên ở giới tính khác có tỉ lệ tham gia khóa học cao hơn ở nữ giới và nam giới nhưng tại sao họ lại có tỉ lệ ở các thành phố phát triển thấp hơn????

Ta cùng tìm hiểu thêm về thêm về kinh nghiệm công việc để rõ hơn

## C. Đưa ra các câu hỏi có ý nghĩa cần trả lời

## D. Tiền xử lý và phân tích dữ liệu để trả lời cho từng câu hỏi

## E. Tổng hợp lại quá trình thực hiện đồ án